In [0]:
!pip install -U PyDrive

    100% |████████████████████████████████| 993kB 7.3MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from pathlib import Path
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
import warnings
from IPython.display import display, Markdown

from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import plotly
plotly.tools.set_credentials_file(username='glarrea', api_key='DIZJYE2ZGzQ5NHiRZ7xK')

import plotly.plotly as py
import plotly.graph_objs as go
import scipy.cluster.hierarchy as hac

#Import authentication libraries

from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

%matplotlib inline
%config IPCompleter.greedy=True
warnings.filterwarnings('ignore')
sns.set()
%matplotlib inline

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
folder_id='1ueCCZhalRvmM2nXLA1gZihsYJHDNqDXr'
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [0]:
# # Auto-iterate through all files in the midi folder.
file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
for file1 in file_list:
  print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

title: loss_per_meter.csv, id: 10dRAVu_u5Bf9RSmZzVE9i9BLVtdGRZDy
title: training_deforested_7x7_roi_8_v3.csv, id: 1_e4ENK7gz1umTLPc0oxrQO3AjcVXOZ1w
title: training_deforested_7x7_roi_6_v3.csv, id: 1Vcfv1OR3BNRbokA8nfj8NJ7qK0ZSGqMy
title: training_deforested_7x7_roi_3_v3.csv, id: 1SlXyVpnD4HtisoAYG0_vQL9QjUaRaSWm
title: training_deforested_7x7_roi_4_v3.csv, id: 1LuBmCNtc4I1pHMd305WRcy_OzUWBsjUB
title: training_deforested_7x7_roi_1_v3.csv, id: 12o5qOpcJnlZ3WSrnh_8e3vA7Bu9qimvv
title: validation_classified_no_rate.csv, id: 1C7R9Ptl0C8-bw_7g1KI1MV-3St6xMhFm
title: validation_classified_no_lonlat.csv, id: 1PVyT-wi1avbFa4bw-NsUn6D3FJetmsur
title: training_deforested_7x7_roi_1.csv, id: 1YB-7bgSzP72H83gGL2lXhA6DhUjuxIwV
title: training_mixed_v26oct.csv, id: 18uZX1BfhIav7HzfwdTVfSdXaqdyP5XQk
title: training_deforested_7x7_withrate_roi.csv, id: 1xqGX7S-_NJkr32aBoPnULrQLMh3kn9B8
title: training_deforested_7x7_with_cluster.csv, id: 1RTo3jp5lEilp2A07CCS0U0QA-xyzB0pK
title: validation_classified, id

In [0]:
# Download all midi files
for file1 in file_list:
  file_id=file1['id']
  file_title = file1['title']
  if '.csv' in file_title:
    drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [0]:
!ls

adc.json
ejepliyo.csv
loss_per_meter.csv
sample_data
training_deforested_7x7_geom.csv
training_deforested_7x7_roi_1.csv
training_deforested_7x7_roi_1_v3.csv
training_deforested_7x7_roi_3_v3.csv
training_deforested_7x7_roi_4_v3.csv
training_deforested_7x7_roi_6_v3.csv
training_deforested_7x7_roi_8_v3.csv
training_deforested_7x7_with_cluster.csv
training_deforested_7x7_withrate_roi.csv
training_mixed_100.csv
training_mixed_28oct_labeled.csv
training_mixed_29oct_10k.csv
training_mixed.csv
training_mixed_v26oct.csv
training_mixed_v28oct_labeled_mdd.csv
training_mixed_v29oct_300m.csv
training_mixed_v29oct_300m_geom.csv
validation_classified.csv
validation_classified_no_lonlat.csv
validation_classified_no_rate.csv


In [0]:
from sklearn import metrics

def print_binary_confusion_matrix(matrix):
    TN = matrix[0,0]
    FN = matrix[1,0]
    FP = matrix[0,1]
    TP = matrix[1,1]

    print ('              +-----------------+')
    print ('              |   Predicción    |')
    print ('              +-----------------+')
    print ('              |    +   |    -   |')
    print ('+-------+-----+--------+--------+')
    print ('| Valor |  +  |  {:5d} |  {:5d} |'.format(TP, FN) )
    print ('| real  +-----+--------+--------+')
    print ('|       |  -  |  {:5d} |  {:5d} |'.format(FP, TN) )
    print ('+-------+-----+--------+--------+')

## Read data

In [0]:
df_1 = pd.read_csv('training_deforested_7x7_roi_1_v3.csv', sep=',')
df_3 = pd.read_csv('training_deforested_7x7_roi_3_v3.csv', sep=',')
df_4 = pd.read_csv('training_deforested_7x7_roi_4_v3.csv', sep=',')
df_6 = pd.read_csv('training_deforested_7x7_roi_6_v3.csv', sep=',')
df_8 = pd.read_csv('training_deforested_7x7_roi_8_v3.csv', sep=',')

datasets = [df_1,df_3,df_4,df_6,df_8]
# datasets = [df_1] #for testing
datasets = list(map(lambda x: x.drop(['carbon_density','system:index', '.geo','cluster_mean'],axis=1), datasets))


## Explore variables

In [0]:
for i in datasets:
    print(np.unique(i['cluster'], return_counts=True))

(array([0, 1]), array([9282, 9996]))
(array([0, 1]), array([9993, 9999]))
(array([0, 1]), array([ 9998, 10000]))
(array([0, 1]), array([ 9958, 10000]))
(array([0, 1]), array([ 9998, 10000]))


## NN

In [0]:
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import os
import time
import itertools


In [0]:
df_1 = pd.read_csv('training_deforested_7x7_roi_1_v3.csv', sep=',')
df_3 = pd.read_csv('training_deforested_7x7_roi_3_v3.csv', sep=',')
df_4 = pd.read_csv('training_deforested_7x7_roi_4_v3.csv', sep=',')
df_6 = pd.read_csv('training_deforested_7x7_roi_6_v3.csv', sep=',')
df_8 = pd.read_csv('training_deforested_7x7_roi_8_v3.csv', sep=',')

datasets = [df_1,df_3,df_4,df_6,df_8]
# datasets = [df_1] #for testing
datasets = list(map(lambda x: x.drop(['carbon_density','system:index', '.geo','cluster_mean'],axis=1), datasets))
df_1 = df_1.drop(['carbon_density','system:index', '.geo','cluster_mean'],axis=1)


In [0]:
test_file = !gsutil ls gs://road_detection/
test_file=test_file[:-1]

In [0]:
driveDataset = tf.data.TFRecordDataset(test_file, compression_type='GZIP')

# Names of the features.
bands =  ['distance_dep','distance_vec','distance_nac','altitude','distance_parks','distance_buffer',
              'distance_villages','latitude','longitude','distance_any']
bands.sort() #to match the order of exported variables
featureNames = list(bands)
PATCH_WIDTH = 256
PATCH_HEIGHT = 256
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

# Feature columns
columns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.int64) for k in featureNames
]
featuresDict = dict(zip(featureNames, columns))
# mean=scaler.mean_
# scale=scaler.scale_
def predict_input_fn(fileNames,mean_,scale_):
  
  # Note that you can make one dataset from many files by specifying a list.
  driveDataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
  
  def normalize(value,mean_,scale_):
    mean =tf.convert_to_tensor(mean_, dtype=tf.float32)
    mean= tf.transpose(tf.reshape(mean, [10,1]),[1,0])

    scale=tf.convert_to_tensor(scale_, dtype=tf.float32)
    scale= tf.transpose(tf.reshape(scale, [10,1]),[1,0])

    new = tf.divide(tf.subtract(value,mean),scale)
    return new
  def parse(example_proto):
    parsed_features = tf.parse_single_example(example_proto, featuresDict)
    altitude=tf.cast(parsed_features['altitude'], tf.float32),
    distance_any=tf.cast(parsed_features['distance_any'], tf.float32),
    distance_buffer=tf.cast(parsed_features['distance_buffer'], tf.float32),
    distance_dep=tf.cast(parsed_features['distance_dep'], tf.float32),
    distance_nac=tf.cast(parsed_features['distance_nac'], tf.float32),
    distance_parks=tf.cast(parsed_features['distance_parks'], tf.float32),
    distance_vec=tf.cast(parsed_features['distance_vec'], tf.float32),
    distance_villages=tf.cast(parsed_features['distance_villages'], tf.float32),
    latitude=tf.cast(parsed_features['latitude'], tf.float32),
    longitude=tf.cast(parsed_features['longitude'], tf.float32),
  #   data=np.array([altitude,distance_any,distance_buffer,distance_dep,
  #             distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude]).reshape(1,10).flatten()
    data = tf.concat([altitude,distance_any,distance_buffer,distance_dep,
              distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude],axis=0)
    data = tf.transpose(data, [1, 0, 2])
    return data
  
  parsedDataset = driveDataset.map(parse, num_parallel_calls=5)
  parsedDataset = parsedDataset.flat_map(lambda features: tf.data.Dataset.from_tensor_slices(features))  
#   parsedDataset = parsedDataset.batch(PATCH_WIDTH * PATCH_HEIGHT)
  iterator = parsedDataset.make_one_shot_iterator().get_next()
  iterator =tf.transpose(iterator, [1,0])
  tensor = normalize(iterator,mean_,scale_)
  return tensor


######################
def parse(example_proto):
  parsed_features = tf.parse_single_example(example_proto, featuresDict)
  altitude=tf.cast(parsed_features['altitude'], tf.float32),
  distance_any=tf.cast(parsed_features['distance_any'], tf.float32),
  distance_buffer=tf.cast(parsed_features['distance_buffer'], tf.float32),
  distance_dep=tf.cast(parsed_features['distance_dep'], tf.float32),
  distance_nac=tf.cast(parsed_features['distance_nac'], tf.float32),
  distance_parks=tf.cast(parsed_features['distance_parks'], tf.float32),
  distance_vec=tf.cast(parsed_features['distance_vec'], tf.float32),
  distance_villages=tf.cast(parsed_features['distance_villages'], tf.float32),
  latitude=tf.cast(parsed_features['latitude'], tf.float32),
  longitude=tf.cast(parsed_features['longitude'], tf.float32),
#   data=np.array([altitude,distance_any,distance_buffer,distance_dep,
#             distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude]).reshape(1,10).flatten()
  data = tf.concat([altitude,distance_any,distance_buffer,distance_dep,
            distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude],axis=0)
#   data = tf.squeeze(tf.transpose(data, [1, 0, 2]))
  data = tf.transpose(data, [1, 0, 2])
  
  return data


def normalize(value,mean_,scale_):
  mean =tf.convert_to_tensor(mean_, dtype=tf.float32)
  mean= tf.transpose(tf.reshape(mean, [10,1]),[1,0])
  
  scale=tf.convert_to_tensor(scale_, dtype=tf.float32)
  scale= tf.transpose(tf.reshape(scale, [10,1]),[1,0])

  new = tf.divide(tf.subtract(value,mean),scale)
  return new

# # Add normalized differences and 3-D coordinates to the dataset.  Shift the label to zero.
# def normalize(example):
#   example['altitude'] = operation_norm(example['altitude'],mean[0],scale[0])
#   return example


In [0]:
driveDataset = tf.data.TFRecordDataset(test_file, compression_type='GZIP')
parsedDataset = driveDataset.map(parse, num_parallel_calls=5)
parsedDataset = parsedDataset.flat_map(lambda features: tf.data.Dataset.from_tensor_slices(features))

foo= parsedDataset.make_one_shot_iterator().get_next()
aa=tf.transpose(foo, [1,0])
aa=normalize(aa, scaler.mean_,scaler.scale_)

#  print(normalize(tensor.eval(),'normie')
with tf.Session() as sess:
  for i in range(3):
    print(aa.eval().shape)
#     testo=sess.run(foo)
#     print(sess.run(foo))
#     print(sess.run(foo))

(1, 10)
(1, 10)
(1, 10)


In [0]:
mean=tf.convert_to_tensor(scaler.mean_, dtype=tf.float32)
mean= tf.transpose(tf.reshape(mean, [10,1]),[1,0])
mean

<tf.Tensor 'transpose_10:0' shape=(1, 10) dtype=float32>

In [0]:
# Batch Normalization func
def batch_norm(x, train=True, name="batch_norm"):
  with tf.variable_scope(name):
    return tf.contrib.layers.batch_norm(x,
                                            decay=0.9,
                                            updates_collections=None,
                                            epsilon=1e-5,
                                            scale=True,
                                            scope=name)
#Linear activation
def dense(input_, output_size, name=None, stddev=0.02, bias_start=0.0, activation='relu'):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(name):
        matrix = tf.get_variable("Weights", [shape[1], output_size], tf.float32,
                                 tf.initializers.glorot_normal(seed=0))
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        linear = tf.matmul(input_, matrix) + bias
        if activation == 'relu':
          return tf.nn.relu(linear, name='activation_relu')
        if activation == 'sigmoid':
          return tf.nn.sigmoid(linear, name='activation_relu')
        if activation == 'logits':
          return linear
        
        
        
#Droputout      
def dropout(input_,name='dropout',mode='train'):
  if mode == 'train':
    return tf.nn.dropout(
                x=input_,
                keep_prob=0.2,
                noise_shape=None,
                seed=0,
                name=name)
  if mode == 'predict':
    return input_

In [0]:
def net_0(x):
  
  d1= dense(x, 256,name='d1')
  d2= dense(d1,128,name ='d2')
  d3 = dense(d2,64, name = 'd3')
  d4 = dense(d3,32, name = 'd4')
  d5 = dense(d4,16, name = 'd5')
  logits = dense(d5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d5,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def net_0_1(x):
  
  d1= dense(x, 1024,name='d1')
  d2= dense(d1,1024,name ='d2')
  d3 = dense(d2,512, name = 'd3')
  d4 = dense(d3,256, name = 'd4')
  d5 = dense(d4,128, name = 'd5')
  d6 = dense(d5,56, name = 'd6') 
#   dp0= dropout(d6,name='dp0')
  logits = dense(d6,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d6,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted



def net_1(x,mode='train'):
  
  d1= dense(x, 512,name='d1')
  bn1 = batch_norm(d1,name='bn1')
  dp0= dropout(bn1,name='dp0')
  d2= dense(dp0,128,name ='d2')
  bn2 = batch_norm(d2,name='bn2')
  dp1= dropout(bn2,name='dp1')
  d3 = dense(dp1,64, name = 'd3')
  bn3 = batch_norm(d3,name='bn3')
  dp2= dropout(bn3,name='dp2')
  d4 = dense(dp2,32, name = 'd4')
  bn4 = batch_norm(d4,name='bn4')
  dp3= dropout(bn4,name='dp3')
  d5 = dense(dp3,16, name = 'd5')
  bn5 = batch_norm(d5,name='bn5')
  logits = dense(bn5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(bn5,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def net_2(x):
  
  d1= dense(x, 60,name='d1')
  d2= dense(d1,15,name ='d2')
  logits = dense(d2,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d2,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def reg_0(x):
  
  d1= dense(x, 120,name='d1')
  bn1 = batch_norm(d1,name='bn1')
  d2= dense(d1,60,name ='d2')
  logits = dense(d2,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d2,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted


In [0]:
#Model preparation


def RUN(sess,X=None,y=None,batch_size=256, epoch=100, learning_rate=0.01, 
        mode='train',name='model_1',test_file=None,mean_=None,scale_=None):
  
  if mode == 'train':
    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3)
    X_tensor= tf.placeholder(tf.float32, shape=(None,X_train.shape[1]),
                                    name='X_tensor')
    y_tensor= tf.placeholder(tf.float32, shape=(None),
                                    name='y_tensor')
  if mode == 'tfrecord':  
    X_train = predict_input_fn(test_file,mean_,scale_)
    
  if mode =='predict':
    X_train = X #is not training, just the name
    y_train = y
    X_tensor= tf.placeholder(tf.float32, shape=(None,X_train.shape[1]),
                                      name='X_tensor')
    y_tensor= tf.placeholder(tf.float32, shape=(None),
                                      name='y_tensor')
  def model(x,y=None,name='model_1'):

    if y != None:
      with tf.variable_scope(name, reuse= tf.AUTO_REUSE):
        logits, predicted = net_0(x)  
        loss = tf.nn.sigmoid_cross_entropy_with_logits(_sentinel=None, 
                labels=y, logits=logits, name='loss') 
        return loss, predicted
    if y == None:
      with tf.variable_scope(name,reuse= tf.AUTO_REUSE):
        logits, predicted = net_0(x)  
      return predicted  
   
  if mode =='train':
    loss,predicted = model(X_tensor, y_tensor,name='model_1')
    opt = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
    t_vars = tf.trainable_variables()
    model_opt=opt.minimize(loss, var_list=t_vars)
    saver=tf.train.Saver()


    start_time = time.time()
    tf.global_variables_initializer().run()
    
    
    for epoch in range(epoch):
      batch_idxs = len(X_train) // batch_size
      for idx in range(0, batch_idxs): 
        batch_X_train = X_train[idx*batch_size:(idx+1)*batch_size]
        batch_y_train = y_train[idx*batch_size:(idx+1)*batch_size]

        sess.run(model_opt, feed_dict={X_tensor: batch_X_train, y_tensor: batch_y_train})

  
      loss_print=tf.reduce_mean(loss)
      classified = tf.to_int32(predicted > 0.5)
      correct_train = tf.equal(classified, y_train.values)
      correct_val = tf.equal(classified, y_val.values)
      
      train_loss = loss_print.eval({X_tensor: X_train, y_tensor: y_train})     
      val_loss = loss_print.eval({X_tensor: X_val, y_tensor: y_val})
      
      train_acc = tf.reduce_mean(tf.cast(correct_train, tf.float32)).eval({X_tensor: X_train, y_tensor: y_train})
      val_acc = tf.reduce_mean(tf.cast(correct_val, tf.float32)).eval({X_tensor: X_val, y_tensor: y_val})
      saver.save(sess, './model/' + 'model.ckpt', global_step=epoch+1)
      print("Epoch: [%2d] time: %4.4f, train_loss: %.8f, val_loss: %.8f, train_acc: %.8f, val_acc: %.8f" \
                  % (epoch, time.time() - start_time, train_loss, val_loss,train_acc,val_acc ))
  if mode =='predict':
  
    if y is not None:
      _,predicted = model(X_tensor, y=y_tensor,name='model_1')
      tf.global_variables_initializer().run()
      saver=tf.train.Saver()
      ckpt = tf.train.get_checkpoint_state('./model/')
      saver.restore(sess, ckpt.model_checkpoint_path)
      
      classified = tf.to_int32(predicted > 0.5)
      correct_prediction = tf.equal(classified, y_train)
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).eval({X_tensor: X_train, y_tensor: y_train})
      print('DA AUQIRASI', accuracy)
      
    if y is None:
      predicted = model(X_tensor, y=None,name='model_1')
      tf.global_variables_initializer().run()
      saver=tf.train.Saver()
      ckpt = tf.train.get_checkpoint_state('./model/')
      saver.restore(sess, ckpt.model_checkpoint_path)
  
      
      classified = tf.to_int32(predicted > 0.5).eval({X_tensor: X_train})
#       print('class_predicted',classified)
      return classified,predicted.eval({X_tensor: X_train})

  if mode =='tfrecord':
   
    tf.global_variables_initializer().run()
    saver=tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('./model/')
    saver.restore(sess, ckpt.model_checkpoint_path)
    predicted = model(X_train, y=None,name='model_1')
   
    for i in range(0,batch_size):
      classified = tf.to_int32(predicted > 0.5)
      predicted_print=predicted.eval()
#     return classified, X_train, X_train


In [0]:
X = df_1.drop('cluster', axis=1)
y = df_1['cluster']
X_trainval, X_test, y_trainval, y_test = train_test_split(X,y,test_size=0.1)
scaler = preprocessing.StandardScaler().fit(X_trainval)
X_trainval = scaler.transform(X_trainval)
X_test = scaler.transform(X_test)

with tf.Session() as sess:
  RUN(X=X_trainval,y=y_trainval,sess=sess,epoch=100)
  print(RUN(X=X_test, sess=sess,y=y_test.values,mode='predict'))

In [0]:
X = df_1.drop('cluster', axis=1)
y = df_1['cluster']
X_trainval, X_test, y_trainval, y_test = train_test_split(X,y,test_size=0.1)
scaler = preprocessing.StandardScaler().fit(X_trainval)
X_trainval = scaler.transform(X_trainval)
X_test = scaler.transform(X_test)

with tf.Session() as sess:
   print(RUN(X=X_test,y=None,sess=sess,mode='predict'))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-100
(array([0, 1, 1, ..., 0, 0, 1], dtype=int32), array([0.04189542, 0.92646724, 0.9990502 , ..., 0.15773933, 0.00643189,
       0.66713613], dtype=float32))


In [0]:

model = RUN(sess=sess,mode='tfrecord',test_file=test_file,batch_size=1,mean_=scaler.mean_,scale_=scaler.scale_)
iterator1, iterator2 = itertools.tee(model, 2)

# Iterate over the predictions, printing the class_ids and posteriors.
for pred_dict in iterator1:
  class_id = pred_dict['class_ids']
  probability = pred_dict['probabilities']
  print class_id, probability

In [0]:
with tf.Session() as sess:
  sess.run(RUN(sess=sess,mode='tfrecord',test_file=test_file,batch_size=4,mean_=scaler.mean_,scale_=scaler.scale_))
  

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-100


TypeError: ignored

In [0]:
  !rm -r ./model/

In [0]:
!ls ./model

ls: cannot access './model': No such file or directory


In [0]:
df_1.head(15)

,altitude,cluster,distance_any,distance_buffer,distance_dep,distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude
0,336,0,24058,4571024,4571024,24058,4571024,2473,18384,-11769,-69427
1,307,0,11241,4576123,11241,19793,4576123,8914,16104,-11121,-69746
2,316,0,1821,4576801,1821,17109,4576801,279,6174,-11032,-69730
3,264,0,11833,30822,42650,11833,32681,3618,15311,-12239,-69019
4,254,0,32274,61280,60047,32274,65494,4084,23081,-12101,-69421
5,318,0,28862,4571929,4571929,28862,4571929,6610,28623,-11656,-69563
6,292,0,278,7582,278,27269,13230,6025,9066,-12699,-70406
7,366,0,7694,0,7694,4565072,7846,25957,11315,-12482,-71194
8,303,0,48580,71393,70259,48580,4568414,11300,32532,-12087,-69573
9,283,0,11725,11629,11785,11725,31744,4978,22125,-12806,-70266


In [0]:
df_1.head(1)

,altitude,cluster,distance_any,distance_buffer,distance_dep,distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude
0,336,0,24058,4571024,4571024,24058,4571024,2473,18384,-11769,-69427


In [0]:
a=tf.constant([0.1,0.2,0.6,0.7])
b = tf.to_int32(a > 0.5)
with tf.Session() as sess:
  print(sess.run(b))

[0 0 1 1]


In [0]:
X_train.head()


,altitude,distance_any,distance_buffer,distance_dep,distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude
10209,255,2606,2530,36285,2606,17430,492,5287,-12853,-70032
7641,594,2065,5105,2065,15515,6926,5723,2461,-13171,-70749
18464,604,1370,0,1370,4560869,13677,598,4046,-12962,-71406
5577,242,220,0,58050,220,8117,1978,16545,-12843,-69673
13938,261,15319,15256,32711,15319,29372,7431,7188,-12743,-70083


In [0]:
with tf.Session() as sess:
  loss, predicted = sess.run(model(X_train.head(1),y_train.head(1)))

AttributeError: ignored

In [0]:
  X = df.drop('cluster', axis=1)
  y = df['cluster']
  X_trainval, X_test, y_trainval, y_test = train_test_split(X,y,test_size=0.1)
  X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3)
  

In [0]:
X = df_1.drop('cluster', axis=1)
y = df_1['cluster']
X_trainval, X_test, y_trainval, y_test = train_test_split(X,y,test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3)
  

In [0]:
type(y_train.values)

numpy.ndarray

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7251964751421694670
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15177412999099472383
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2957156692381048139
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 363400024912358664
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
df = pd.read_csv('training_deforested_7x7_roi_1_v3.csv', sep=',')
print(df.shape)
df=df.drop(['carbon_density','system:index', '.geo','cluster_mean'],axis=1)
df.head()

(19278, 15)


,altitude,cluster,distance_any,distance_buffer,distance_dep,distance_nac,distance_parks,distance_vec,distance_villages,latitude,longitude
0,336,0,24058,4571024,4571024,24058,4571024,2473,18384,-11769,-69427
1,307,0,11241,4576123,11241,19793,4576123,8914,16104,-11121,-69746
2,316,0,1821,4576801,1821,17109,4576801,279,6174,-11032,-69730
3,264,0,11833,30822,42650,11833,32681,3618,15311,-12239,-69019
4,254,0,32274,61280,60047,32274,65494,4084,23081,-12101,-69421


In [0]:
X_train_val = df.drop('cluster', axis=1)
y_train_val = df['cluster']

In [0]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train_val)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
scaler.mean_

array([ 3.69526092e+02,  9.03138469e+03,  1.04036646e+06,  5.04651152e+05,
        2.90258150e+05,  1.07262323e+06,  5.32368093e+03,  1.20618135e+04,
       -1.24781170e+04, -6.98958841e+04])

In [0]:
scaler.scale_

array([2.68042245e+02, 8.51001632e+03, 1.90735621e+06, 1.39593698e+06,
       1.08746736e+06, 1.92218549e+06, 5.40077248e+03, 8.02563626e+03,
       6.19416699e+02, 6.80591579e+02])

In [0]:
X_scaled = scaler.transform(X_train_val)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_train_val, test_size=0.3, random_state=0)


In [0]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9
0,-0.419807,4.117573,-0.508651,-0.312094,-0.226385,-0.518216,0.390374,2.004350,0.655644,0.533189
1,-0.207154,-0.374310,1.852720,-0.324548,-0.261536,1.821646,-0.435990,-1.308533,1.782511,0.824113
2,-0.423538,-0.936236,-0.544892,-0.337553,-0.265934,-0.550546,-0.674289,-1.281869,-0.647194,-0.232321
3,-0.251923,-0.652688,1.852926,-0.333149,-0.263715,1.821850,-0.623000,-0.937099,1.863232,0.608124
4,-0.550384,-0.895578,-0.541152,-0.353335,-0.265615,-0.550203,-0.676326,0.104189,-0.256504,0.878477


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras import regularizers 


model = Sequential([
#     Dense(1024, activation='relu', input_shape=(10,)),
#     BatchNormalization(),
#     Dense(512, activation='relu',input_shape=(10,)),
    Dense(256, activation='relu',input_shape=(10,)),
    BatchNormalization(),
    Dropout(rate=0.1),
    Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.1)),
    Dropout(rate=0.1),
    Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.1)),
    Dropout(rate=0.1),
    Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.1)),
    #Dense(16, activation='relu'),
    #Dense(8, activation='relu'),
    Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.1)),
    Dropout(rate=0.1),
#     Dense(128, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu'),
#     Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')])

Using TensorFlow backend.


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [0]:
model.compile(loss='binary_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               2816      
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
__________

In [0]:
log = model.fit(X_train, y_train, epochs=100, batch_size=256, validation_data=[X_val, y_val])

Train on 13494 samples, validate on 5784 samples
Epoch 1/100
13494/13494 [==============================] - 1s 50us/step - loss: 0.5599 - acc: 0.7592 - val_loss: 0.5436 - val_acc: 0.7593
Epoch 2/100
13494/13494 [==============================] - 1s 50us/step - loss: 0.5571 - acc: 0.7619 - val_loss: 0.5739 - val_acc: 0.7450
Epoch 3/100
13494/13494 [==============================] - 1s 52us/step - loss: 0.5665 - acc: 0.7557 - val_loss: 0.5553 - val_acc: 0.7597
Epoch 4/100
13494/13494 [==============================] - 1s 49us/step - loss: 0.5629 - acc: 0.7577 - val_loss: 0.5608 - val_acc: 0.7588
Epoch 5/100
13494/13494 [==============================] - 1s 50us/step - loss: 0.5609 - acc: 0.7591 - val_loss: 0.5701 - val_acc: 0.7540
Epoch 6/100
13494/13494 [==============================] - 1s 51us/step - loss: 0.5634 - acc: 0.7568 - val_loss: 0.5607 - val_acc: 0.7561
Epoch 7/100
13494/13494 [==============================] - 1s 49us/step - loss: 0.5666 - acc: 0.7577 - val_loss: 0.5437 - v

In [0]:
def show_results(log):
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(log.history['val_loss'], label='validation')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['acc'], label='train')
    ax2.plot(log.history['val_acc'], label='validation')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('accuracy')
    for ax in axes: ax.legend()
show_results(log)

In [0]:
show_results(log)

In [0]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(X_val).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_val, y_pred_keras)

In [0]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)
auc_keras

In [0]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()